In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning, module="joblib")
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
warnings.filterwarnings('ignore', category=UserWarning, message="Line Search failed")

In [ ]:
#load data and preprocess
data = pd.read_csv('final_result1.csv').fillna(0)
print(data.head())
print(data.isnull().sum())

In [ ]:
#feature selection
features = data[['baseMean', 'log2FoldChange', 'pvalue']]
targets = {'NSCLC': data['NSCLC'], 'SCLC': data['SCLC']}

In [ ]:
#EDA
#histograms and correlation matrix
features.hist(bins=15, figsize=(15, 6), layout=(2, 2))
plt.show()
sns.heatmap(features.corr(), annot=True)
plt.show()

In [ ]:
#SMOTE and data splitting
def split_and_resample(features, target, test_size=0.4, random_state=42):
    x_train, x_temp, y_train, y_temp = train_test_split(features, target, test_size=test_size, random_state=random_state)
    smote = SMOTE()
    x_train_res, y_train_res = smote.fit_resample(x_train, y_train)
    x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=random_state)
    return x_train_res, x_test, x_val, y_train_res, y_test, y_val

data_nsclc = split_and_resample(features, targets['NSCLC'])
data_sclc = split_and_resample(features, targets['SCLC'])

In [ ]:
#random forest pipeline
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),  
    ('rf', RandomForestClassifier(class_weight='balanced', n_jobs=-1))  #n_jobs=-1 uses all available cores
])

#parameter grid for random forest
param_grid_rf = {
    'rf__n_estimators': [100, 200, 300],  
    'rf__criterion': ['gini', 'entropy'],  
    'rf__max_depth': [None, 10, 20],  
    'rf__min_samples_split': [2, 5, 10],  
    'rf__min_samples_leaf': [1, 2, 4],  
    'rf__max_features': ['sqrt', 'log2'],  
    'rf__max_leaf_nodes': [None, 50, 100],  
    'rf__min_impurity_decrease': [0.0, 0.1, 0.2],  
    'rf__bootstrap': [True, False],  
    'rf__oob_score': [False, True],  
    'rf__random_state': [None, 42, 123],  
    'rf__verbose': [0, 1],  
    'rf__warm_start': [False, True],  
    'rf__class_weight': [None, 'balanced'],  
    'rf__ccp_alpha': [0.0, 0.1, 0.2],  
    'rf__max_samples': [None, 0.7, 0.8]  
}


In [ ]:
#function for GridSearchCV and model evaluation
def evaluate_random_forest(data, param_grid):
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='f1', cv=5, verbose=1, n_jobs=-1)
    grid_search.fit(data[0], data[3])  #fit on training data
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    best_rf = pipeline.set_params(**best_params)
    best_rf.fit(data[0], data[3])  #refit on training data
    y_val_pred = best_rf.predict(data[2])  #predict on validation data
    report = classification_report(data[5], y_val_pred)
    return best_params, best_score, report

In [ ]:
#evaluate for NSCLC
best_params_nsclc, best_score_nsclc, report_nsclc = evaluate_random_forest(data_nsclc, param_grid_rf)
print("Best Parameters for NSCLC:", best_params_nsclc)
print("Best F1 Score for NSCLC:", best_score_nsclc)
print("Classification Report for NSCLC (Validation Data):\n", report_nsclc)

In [ ]:
#evaluate for SCLC
best_params_sclc, best_score_sclc, report_sclc = evaluate_random_forest(data_sclc, param_grid_rf)
print("Best Parameters for SCLC:", best_params_sclc)
print("Best F1 Score for SCLC:", best_score_sclc)
print("Classification Report for SCLC (Validation Data):\n", report_sclc)